<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Физическая и логическая память](#toc1_)    
  - [Загрузчик Multiboot](#toc1_1_)    
    - [Формат карты памяти](#toc1_1_1_)    
    - [Пример week2/physmem-map](#toc1_1_2_)    
- [Логическое адресное пространство](#toc2_)    
  - [Процесс](#toc2_1_)    
  - [Пример week2/log_mem/fork](#toc2_2_)    
- [Как логические адреса отображаются на физические](#toc3_)    
- [Сегментация](#toc4_)    
    - [Таблица дескрипторов сегментов](#toc4_1_1_)    
    - [Селектор сегмента](#toc4_1_2_)    
    - [Пример week2/log_mem/loop](#toc4_1_3_)    
    - [Пример week2/log_mem/kern](#toc4_1_4_)    
    - [Global Descriptor Table](#toc4_1_5_)    
    - [Дескриптор сегмента в защищенном режиме (запись в GDT)](#toc4_1_6_)    
    - [Преобразование в физический адрес](#toc4_1_7_)    
    - [Сегментация в Long Mode (64 бита)](#toc4_1_8_)    
- [Страничная организация памяти (paging)](#toc5_)    
    - [Таблица страниц (цифровое дерево)](#toc5_1_1_)    
    - [Translation Lookaside Buffer](#toc5_1_2_)    
    - [Page Fault](#toc5_1_3_)    
  - [Станичная организация памяти в x86 Long Mode (64 bit)](#toc5_2_)    
    - [Размер страницы памяти (4 Кб, 2 Мб, 1 Гб)](#toc5_2_1_)    
    - [Запись в таблице страниц](#toc5_2_2_)    
    - [Задача](#toc5_2_3_)    
    - [Логическое и физическое адресное пространства:](#toc5_2_4_)    
    - [Понятие процесса:](#toc5_2_5_)    
    - [Сегментация и страничная адресация памяти:](#toc5_2_6_)    
- [Алгоритмы аллокации памяти](#toc6_)    
    - [Выравнивание](#toc6_1_1_)    
  - [Простой подход к аллокации](#toc6_2_)    
    - [Аллокация свободного блока](#toc6_2_1_)    
    - [Освобождение занятого блока](#toc6_2_2_)    
    - [Боремся с фрагментацией](#toc6_2_3_)    
    - [Граничные маркеры](#toc6_2_4_)    
- [Задача](#toc7_)    
  - [Buddy аллокатор](#toc7_1_)    
    - [Дескрипторы страниц](#toc7_1_1_)    
    - [Списки свободных блоков](#toc7_1_2_)    
    - [Инициализация](#toc7_1_3_)    
    - [Аллокация](#toc7_1_4_)    
    - [Освобождение](#toc7_1_5_)    
  - [Кеширующий аллокатор.](#toc7_2_)    
- [ SLAB аллокатор](#toc8_)    
  - [Заметки по SLAB-аллокатору](#toc8_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Физическая и логическая память](#toc0_)

Логическая память - как видит память программа.  
Физическая память - как процессор видит память  

    - логические адреса отображаются на физические;
    - все, что соблюдает интерфейс, выглядит как физическая память для CPU

Не все физические адреса соответствуют памяти  
    
    - видеобуфер и другие устройства;
    - "дыры"

Физическое адресное пространство может быть устроено довольно сложно. Нужна карта памяти  

Откуда брать карту физической памяти?  
 
    - из документации аппаратной платформы;
    - спросить BIOS/UEFI/etc:
        BIOS: прерывание 0x15, команда 0xe820;
        UEFI: функция GetMemoryMap;
    - спросить загрузчик
        GRUB поддерживает спецификацию multiboot.

## <a id='toc1_1_'></a>[Загрузчик Multiboot](#toc0_)

Спецификация этого режима определяет требования к файлу, чтобы он индетифицировался как ядро ОС (OS image format).   
Multiboot header must be contained completely within the first 8192 bytes of the OS image, and must be longword (32-bit) aligned. Обязательные поля magic, flags (устанавливают режимы, которые ядро будет требовать у загрузчика) и checksum (число, которое в сумме с magic + flags дает 0). Первое поле magic должно равняться в точности 0x1BADB002. Остальные поля заголовка ядра устанавливаются в зависмости от выставленных flags.  


Кроме того, за загрузчиком предусматривается обязанность создания специальной структуры в памяти (Multiboot information data structure), физический адрес которой сохраняется в регистре EBX процессора, и к которой процессор может обратиться когда необходимо. Обязательным здесь является только поле flags (32 бита), в зависимости от корого будут рассматриваться как валидные (инициализированные) остальные поля этой структуры.    

В этой структуре среди прочего, если стоит соответствующий флаг (6й), должна содержаться карта памяти. Кроме этого, в ней могут быть адреса загружаемых драйверов, и куча других конфигов. Если загрузчик (по каким-то причинам) не получил карту памяти, то он не поставит этот флаг в Multiboot information data structure. А если все ОК, то в полях mmap_length
и mmap_addr будут соответственно размер карты памяти и физ.адрес в памяти, где она лежит.   

### <a id='toc1_1_1_'></a>[Формат карты памяти](#toc0_)

Это массив структур mmap_entry (начало участка памяти from, размер участка памяти to, тип type (доступна для размещения данных или зарезервирована, например, как память устройств или просто "дыры"))
Другими словами, карта памяти mmap_addr - маппинг физической памяти, это "отфильтрованный" список доступной к использованию физ. памяти

### <a id='toc1_1_2_'></a>[Пример week2/physmem-map](#toc0_)
В примере physmem-map собрано такое мини-ядро ОС, которое эмулятор qemu определяет как мультибут и, соответственно, для него qemu заполняет память, инициализирует регистры, готовит ему описанные выше структуры данных для загрузки. Ядро из примера умеет разбирать, то, что заготовил загрузчик (в данном случае эмулятор qemu), пушит адрес карты памяти в EAX, работает с видеобуфером и выводит туда информацию о карте памяти в памяти. Ассемблерная часть примера bootstrap.S включает:
    
    - подготовку GDT (глобальной таблицы дескрипторов), 
    - обнуление 0го адреса физической памяти (спецификация Intel, обращение по 0 адресу, как валидному, чревато ошибками)
    - разметку страничного режима в памяти
    - перевод процессора в защищенный режим
    - вызов С кода main()

Все остальное (работа с видеобуфером, преобразование физических адресов в логические, чтение и форматирование данных и т.д.) запрограммировано в С.

\*) Преобразование физических адресов в логические тут просто прибавление константы: /* First address after "canonical hole", beginning of the middle mapping. */
#define HIGHER_BASE	0xffff800000000000


    

# <a id='toc2_'></a>[Логическое адресное пространство](#toc0_)

    - это абстракция - приложение не знает о структуре физической памяти (где есть "дыры" и уже используемые места); 
    - изоляция и защита - каждое приложение имеет свое логическое адресное пространство

Цель - представление физической памяти как последовательный набор "ячеек", которые можно пронумеровать от 0 до n.  

## <a id='toc2_1_'></a>[Процесс](#toc0_)

Это абстрация - контейнер ресурсов ОС.

    ОС может и не поддерживать такую абстрацию как процессы (XX-DOS);
    ОС выделяет ресурсы, например, память, процессам;
    код, исполняющийся в рамках процесса, использует его ресурсы

Процессы, по-умолчанию, изолированны друг отдруга:

    - у каждого процесса свое логическое адресное пространство;
    - т. е. код в рамках одного процесса не может залезть в память другого процесса (если речь не об отладчиках или ряде других категорий процессов).

## <a id='toc2_2_'></a>[Пример week2/log_mem/fork](#toc0_)

В /fork/main() вызывается функция fork(), порождающая дочерний процесс как копию родительского. После чего проверяется pid процесса, по которому определяется это дочерний или родительский процесс. Сначала дочерний процесс выводит свой (логический, т.к. все это уже под управлением ОС) адрес в памяти, родительсткий ждет его завершения и выводит свой адрес:

    $ cc main.c
    $ ./a.out
    value (0x55f8f920c010) = 43
    value (0x55f8f920c010) = 42
*) логический адрес одинаковый, процессы разные, т.е. одинаковый лог.адрес отображается на различные участки физической памяти. fork() make two identical copies of address spaces, one for the parent and the other for the child. За то, чтобы они не пересекались отвечает ОС.  
**) любой процесс в linux это fork() существующего + exec(). Внешне просто, под капотом, не так просто. В windows этим занимается CreateProcess, который более сложен (и хрупок) изначально. Альтернативный подход разрабатывал Гугл в ОС Фуксия, где процесс копируется от существующего, но не запускается, потом конфигурируется, потом запускается.



# <a id='toc3_'></a>[Как логические адреса отображаются на физические](#toc0_)
Два основных способа: 
    
    сегментация (важно для x86);
    paging (страничная организация памяти).

# <a id='toc4_'></a>[Сегментация](#toc0_)

Сегментация в Real Mode:
    
    логический адрес - сегмент (SEG) и смещение (OFF);
    SEG хранится в одном из сегментных регистров (CS,SS,DS,ES,GS,FS);
    Aphys = (SEG×16+OFF) mod 2**20

Таким образом:

    сегмент SEG начинается по физическому адресу SEG×16;
    сегмент SEG имеет размер 2**16 байт (64 Кб).
    
Но допустим, что SEG - это идентификатор сегмента физической памяти. А что если разрешить ОС изменять параметры (начало и размер) сегмента?
### <a id='toc4_1_1_'></a>[Таблица дескрипторов сегментов](#toc0_)

Таблица строится ОС и состоит из двух полей (база сегмента - где он начинается, размер сегмента). Тогда физическая память отображается в разного размера сегменты, часть из которых можно резервировать для разных процессов/устройств, например VGA буфер. Идентификатором сегмента является база, по которой он и ищется в таблице.  
Пусть ОС "выдает" каждому процессу свой дескриптор (SEG). 
    
    каждый дескриптор описывает свой участок физической памяти;
    разные процессы пользуются разными дескрипторами (т. е. разные значения SEG);
    непривилегированному коду запрещено изменять сегментные регистры.

Именно так организована сегментация памяти в защищенном режиме х86 (32 бита, до 4 Гб адресуемой памяти).

### <a id='toc4_1_2_'></a>[Селектор сегмента](#toc0_)

В старших 13 битах он содержит индекс дескриптора в дескрипторной таблице (т.е. максимум 2**13=8192 сегментов), 1 бит (поле TL (Table level)) определяющий, к какой дескрипторной таблице производится обращение (LDT или GDT), а также запрашиваемые права доступа к сегменту в последних (младших) 2 битах (поля CPL/RPL или Current/Requested Privilege Level). Соответственно уровней привелений в х86 может быть только 4:  

    ring0 - ring3, где ring0 - наивысший уровень привилегий (код ядра ОС), а ring3 - низший уровень привилегий (код пользовательских приложений).

*) в сегментных регистрах CS, SS содержатся селекторы для текущих уровней привилегий, в остальных DS ES GS FS - для запрашиваемых уровней привелегий.

### <a id='toc4_1_3_'></a>[Пример week2/log_mem/loop](#toc0_)

main.c это просто бесконечный цикл. Компилируем, запускаем отладчиком gdb, смотрим регистры.

    $ cc main.c
    $ gdb ./a.out                                       # запустили в обычном, непривилегированном режиме
    (gdb) r                                             # run (цикл запустился)            
    ^C                                                  # ctrl + c
    Program received signal SIGINT, Interrupt.
    (gdb) info registers
    ...
    cs             0x33                51               # 0000 0000 0011 0011, т.е. индекс сегмента =b110, таблица дескрипторов глобальная TL=0, CPL=b11 уровень ring3 (пользовательский)
    
### <a id='toc4_1_4_'></a>[Пример week2/log_mem/kern](#toc0_)

getseg.c - код мини-модуля ядра, который должен работать в привилегированном режиме. Он в ходе работы сбросит в лог содержимое регистра CS.

    $ make
    $ sudo insmod getseg.ko
    $ dmesg
    ...
    [114142.495466] CS = 0x10                           # 0000 0000 0001 0000, т.е. индекс сегмента =b010, таблица дескрипторов глобальная TL=0, CPL=b00 уровень ring0 (ядро ОС)
    $ sudo rmmod getseg
    

### <a id='toc4_1_5_'></a>[Global Descriptor Table](#toc0_)
В x86 таблицу дескрипторов называют GDT. В узком смысле она не отличается от LDT:
    
    адрес и размер GDT хранятся в специальном регистре GDTR;
    писать и читать GDTR можно с помощью инструкций LIDT и SIDT;
    писать в GDTR может только привилегированный код.

### <a id='toc4_1_6_'></a>[Дескриптор сегмента в защищенном режиме (запись в GDT)](#toc0_)

Это 64 битная структура данных в которой 32 бита отведено под базу (до 4 Гб адресации), 20 бита - под т.н. лимит (размер сегмента), еще 12 бит - различные флаги.  

### <a id='toc4_1_7_'></a>[Преобразование в физический адрес](#toc0_)

На входе у нас селектор сегмента (указывает на запись в GDT) и логический адрес ("эффективный адрес" по спецификации). Если логический адрес больше лимита, указанного в записи GDT, то генерируется исключение. Иначе, к логическому адресу добавляется база из записи GDT - получается физический адрес ("линейный" адрес в спецификации интел).

### <a id='toc4_1_8_'></a>[Сегментация в Long Mode (64 бита)](#toc0_)

Сегментация в Long Mode практически не используется

    ES,DS,FS,GS обычно равны 0;
    поля Base и Limit дескрипторов игнорируются;
    CS и SS все еще хранят CPL.

Вместо сегментации используется страничная организация памяти.

# <a id='toc5_'></a>[Страничная организация памяти (paging)](#toc0_)

Суть метода в использовании словаря, который хранит отображение логических адресов на физические. Ядро ОС создает свой словарь для каждого процесса.  
Словарь отображения памяти:  

    отображать каждый байт отдельно непрактично;
    отображение происходит блоками фиксированного размера (страницами);
    размер страницы определяется архитектурой (типичные размеры: 4Kb и 64Kb);
    несколько областей логической памяти могут отображаться на одну область фазической;
    в логической памяти могут быть пропуски;
    более старшие области логической памяти могут отображаться на более младшие области физической.
    

Как должен выглядеть словарь?

    не каждый процесс использует все логическое адресное пространство (даже в 32-битных системах и тем более в 64-битных);
    не хочется хранить информацию для неиспользуемых страниц;
    структура должна быть сравнительно простой (хеш-таблица или дерево поиска слишком сложные, чтобы реализовать их на процессоре для отображения памяти).

### <a id='toc5_1_1_'></a>[Таблица страниц (цифровое дерево)](#toc0_)

Логический адрес разбивается на две части (номер(а) страницы в старших битах + смещение в младших).

    первый номер страницы является индексом в таблице первого уровня (корневой), по которому записан индекс для таблицы следующего уровня и т.д. до level0
    в таблице level0 номер физической страницы памяти, в которой по смещению расположены интересующие нас данные;
    число уровней таблиц страниц определяетя архитектурой.

\*) 1 байт определяется как минимально адресуемая единица памяти. Т.е. в общем случае 1 байт != 8 бит (например, в стандартах C и С++ CHAR_BIT >= 8). Искать причину почему 1 байт это минимально адресуемая единица памяти не имеет смысла, если 1 байт определен как минимально адресуемая единица памяти. Искать причину почему 1 байт это 8 бит можно, но с практической точки зрения не очень полезно.

Проблемка - для одного обращения к физической памяти процессору нужно n-раз обратиться к таблицам страниц (по числу уровней), работа с памятью становится в n раз медленнее.
Решение - кеширование обращений.

### <a id='toc5_1_2_'></a>[Translation Lookaside Buffer](#toc0_)

TLB может значительно ускорить обращение к памяти, если код не обращается каждый раз к новой странице.
Процессор не может отследить изменения в таблицах страниц:
    
    TLB не прозрачен, т. е. необходимо явно "сбрасывать" записи (кэш) при изменениях в таблицах страниц;
    об этом тоже должно заботиться ядро ОС.

### <a id='toc5_1_3_'></a>[Page Fault](#toc0_)

Не все записи в таблицах страниц реально используются. Что произойдет, если предпринимается попытка разыменовать произвольное число в качестве адреса.  
Исключение Page Fault генерируется, когда в таблице страниц ищется логический адрес, которого там нет.  
Кроме того, Page Fault генерируется:

    когда предпринимается попытка записи по логическому адресу (который есть), для которого установлен флаг защиты от записи;
    когда предпринимается исполнить код, записанный по логическому адресу (который есть), для которого установлен флаг запрета на исполнение;
    когда предпринимается попытка прочитать из области логической памяти процессом, чьи привилегии не достаточны для этой операции (процесс будет завершен).

## <a id='toc5_2_'></a>[Станичная организация памяти в x86 Long Mode (64 bit)](#toc0_)

Используется 4-х уровневая таблица страниц. 
Логический 64 битный адрес включает:

    4 9-битных указателя страниц (PML4, Directory Ptr, Directory, Table) - всего 36 бит;
        соответсвующие таблицы называются PML4E (корневая), PDTE, PDE, PTE
        всего в каждой таблице 2**9 = 512 записей, размер таблицы установлен в 4 Кб.
        все таблицы выравниваются по 4096 байт (адрес начала таблицы делится нацело на 4096)
        т.е. каждая запись - 8 байт (64 бита)
    12 битное смещение offset (т.е. размер страниц памяти 2**12 = 4096 = 4 Кб)
    итого 36 + 12 = 48 бит
        старшие биты логического адреса (выше 47 бита) не используются, они устанавливаются равными 47-му биту
        *) старшие биты иногда могут использоваться в отдельных технологиях (tagged pointer), на которых строятся lock-free структуры данных и алгоритмы (которые не требуют синхронизации). 
        Правда эти же алгоритмы прекрасно могут быть реализованы и без старших битов, поэтому они по большому счету, обычно вообще не используются.

48 битный логический адрес позволяет адресовать 2**48 = 260 Тб памяти, что на порядки перекрывает современные потребности

Адрес корневой таблицы хранится в спец.регистре CR3.  

### <a id='toc5_2_1_'></a>[Размер страницы памяти (4 Кб, 2 Мб, 1 Гб)](#toc0_)

В описанном выше виде (по умолчанию) размер страницы 4 Кб. 
Вместе с тем, во второй (PDTE) или третьей (PDE) таблице в записях может быть установлен специальный бит (PS - page size), который говорит о том, что следующая часть логического адреса, это не соответствующие таблицы, а такое расширенное смещение.  
В первом случае смещение получится размером 12 + 9 + 9 бит = 30 бит => page size = 2\**30 = 1 Gb  
Во втором случае offset = 12 + 9 = 21 бит => page size = 2\**21 = 2 Mb  

### <a id='toc5_2_2_'></a>[Запись в таблице страниц](#toc0_)
Основные поля 64 битной записи (биты, имя, описание):

    63 XD если 1, то запрещено исполнение
    51-12 сам физический адрес (или ссылка на запись в следующей таблице) - выравнивается по размеру смещения (младшие биты по размеру смещения =0)
    7 PS (page size) если 1, то это последний уровень, дальше идет смещение
    2 U/S если 0, то запрещен непривилегированный доступ (непривилиг.код - у которого в TPL (младшие 2 бита регистра CS) что-то оличное от 0)
    1 R/W если 0, то запись запрещена
    0 P (бит присутствия) если 0, запись не используется (=> Page Fault)
     

    


### <a id='toc5_2_3_'></a>[Задача](#toc0_)
Проверим, как вы поняли paging. Для этого вам предлагается выступить в качестве процессора и преобразовать несколько логических адресов в физические. Формат входных данных следующий:

    в первой строке вам даны 3 числа m,q,r≥0 , где q - это количество запросов, на которые вам нужно ответить, r - физический адрес корневой таблицы страниц
    следующих m строках записаны пары paddr и value - описание физической памяти, каждая пара значит, что по физическому адресу paddr хранится 64 битное значение value, при этом гарантируется, что все paddr различны, выровнены на границу 8 байт и помещаются в 64 бита
    в последних q строках идут целые числа - логические адреса, которые вам нужно преобразовать в физические, для каждого из этих чисел нужно вывести на отдельной строке либо физический адрес, либо слово "fault", если преобразовать логический адрес в физический нельзя.

Считайте, что таблица страниц имеет формат 64 битного режима x86 (4 уровня, каждая страница 4 KB, каждая запись 8 байт, формат записи был показан в лекциях), но вы можете игнорировать все поля, кроме бита присутствия (на картинке бит P - нулевой бит) и собственно физического адреса.

Для всех физических адресов, не указанных во входных данных (среди m пар paddr value), считайте, что по этим адресам хранятся нули.

ВАЖНО: это было неочевидно из видео, но все физические адреса, которые хранятся в записях таблицы страниц должны быть выровнены, как минимум, на границу 4096 байт (4Kb), т. е. младшие 12 бит физических адресов всегда равны 0, соответственно, хранить младшие биты нет смысла и в записе таблицы страниц они не хранятся - их место занимают специальные флаги. Убедитесь, что вы понимаете приведенный пример.

In [237]:
# ВЕРНО! (без битовой арифметики, чисто на 01-строках)

mem_struct = {}

def b64(i: int) -> str:
    """ Переводит int в 64 битную двоичную строку.
    Уот так уот...
    """
    return ('0'*(64 - len(format(i, 'b'))) + format(i, 'b'))

def parse_laddr(log_addr: str) -> str:
    """ Разбирает двоичную 64 битную строку на компоненты 
    логического адреса
    """
    PML4 = log_addr[-48:-39]
    PDTE = log_addr[-39:-30]
    PDE = log_addr[-30:-21]
    PTE = log_addr[-21:-12]
    offset = log_addr[-12:]
    return (PML4, PDTE, PDE, PTE, offset)

def get_paddr(log_addr: str, cr3: str, fout=sys.stdout):
    """ Получает значения из "памяти" по таблицам компонентов 
    логического адреса.
    """
    value = cr3
    *idxs, offset = parse_laddr(log_addr)
    for idx in idxs:
        int_val = int(value, base=2) + int(idx, base=2) * 8         # физ.адр. следующей таблицы + индекс*8 (смещение от начала таблицы * 8 "байт" (размер записи))
        value = mem_struct.get(b64(int_val), b64(0))
        if value[-1] == '0':                                        # бит присутствия P
            print('fault', file=fout)
            return
        elif value[-8] == '1':                                      # бит последней таблицы PS
            offset = "".join(idxs[idxs.index(idx) + 1:]) + offset   # все, что после текущей таблицы - смещение
            break
        value = value[-52:-12] + '0'*12                             # физ.адрес следующей таблицы, выровненный по минимальному offset 2**12
    print(int(value, base=2) + int(offset, base=2), file=fout)

def main(data, out):

    with open(data, 'r') as fin, open(out, 'w') as fout:
        mem_rows, queries, cr3 = map(int, fin.readline().split())
        for _ in range(mem_rows):
            paddr, value = map(int, fin.readline().split())
            mem_struct[b64(paddr)] = b64(value)

        for _ in range(queries):
            log_addr = b64(int(fin.readline()))
            get_paddr(log_addr, b64(cr3), fout=fout)

# main(data = "test.txt", out = "res.txt")
main(data = "dataset_44327_15.txt", out = "res.txt")

In [12]:
def parse_laddr(laddr):
    """ Разбирает лог.адрес на компоненты 
    """
    pml4 = (laddr >> 39) & 0x1ff        # младшие 9 бит (маске 0x1ff = 0b111111111) у сдвинутого вправо на 39 бит значения
    dir_ptr = (laddr >> 30) & 0x1ff     # ... на 39 бит значения
    directory = (laddr >> 21) & 0x1ff   # ...
    table = (laddr >> 12) & 0x1ff       # ...
    offset = laddr & 0xfff              # И по маске 0xfff (12 единиц или 0b111111111111), т.е. результат - младшие 12 бит

    return (pml4, dir_ptr, directory, table, offset)

def get_paddr(laddr, mem_struct: dict, cr3, fout=sys.stdout):
    """ Получает значения из "памяти" по таблицам компонентов 
    логического адреса.
    В задаче не задействуется случай с размером страницы, отличным от 4 Кб,
    поэтому проверку бита PS не делаем
    """
    value = cr3
    *idxs, offset = parse_laddr(laddr)
    for idx in idxs:
        index =  idx * 8                            # индекс * 8 (смещение от начала таблицы * 8 "байт" (размер записи))
        value = mem_struct.get(value + index, 0)
        if value & 1 == 0:                          # младший бит - бит присутствия P
            print('fault', file=fout)
            return
        value = value & (0xffffffffff << 12)        # маска = 40 единичек + 12 нулей => обнуляется 12 младших бит(по минимальному offset 2**12) в 52 битном адресе
    print(value + offset, file=fout)

def main(data, out):
    mem_struct = {}

    with open(data, 'r') as fin, open(out, 'w') as fout:
        mem_rows, queries, cr3 = map(int, fin.readline().split())
        for _ in range(mem_rows):
            paddr, value = map(int, fin.readline().split())
            mem_struct[paddr] = value
        for _ in range(queries):
            laddr = int(fin.readline())
            get_paddr(laddr, mem_struct, cr3, fout=fout)
    return 0

%timeit main(data = "dataset_44327_15.txt", out = "res1.txt")

281 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Мой код на C++ в 2,4 раза медленнее, чем код на Python:

    $ time /home/user1/projects/stepik.org/OS/2.Memory/memory_ref

    real    0m0,675s
    user    0m0,660s
    sys     0m0,013s

### <a id='toc5_2_4_'></a>[Логическое и физическое адресное пространства:](#toc0_)
    
    программы используют логические адреса(указатели);
    процессор использует физические адреса;
    ОС определяет как логические адреса отображаютсяна физические.

### <a id='toc5_2_5_'></a>[Понятие процесса:](#toc0_)
    
    каждый процесс имеет свое логические адресное пространство; 
    процессы изолированы друг от друга.

### <a id='toc5_2_6_'></a>[Сегментация и страничная адресация памяти:](#toc0_)

    ОС использует эти аппаратные механизмы для организации изоляции процессов;
    многие современные архитектуры с поддержкой защиты памяти используют paging (и очень немногие сегментацию).


# <a id='toc6_'></a>[Алгоритмы аллокации памяти](#toc0_)

Рассмотрим простейшую постановку задачи:
    
    есть непрерывный участок логической памяти (даны начало участка и его размер, отображение на физ.память не важно);
    функция аллокации: void ∗alloc (int size) - возвращает указатель на свободное место в лог.памяти размером не менее size;
    функция освобождения: void free (void ∗free) - возвращает аллокатору память по указателю, с тем, чтобы аллокатор мог вызать ее по другим запросам.
        типа malloc/free в С, new/delete в С++

### <a id='toc6_1_1_'></a>[Выравнивание](#toc0_)

Некоторые процессоры требуют выравненных указателей. Выровненный (aligned) по x указатель - чье значение делится нацело на х.  
Часто используется "натуральное" выравнивание (по ближайшим степеням 2ки). Выравнивание по иным основаниям 3, 5 ...  :

    для данных объемом <=2 байта - выравнивание 2 байта;
    <= 4 байта - выравнивание 4 байта;
    и т.д.

## <a id='toc6_2_'></a>[Простой подход к аллокации](#toc0_)

Создадим (двух-)связный список свободных блоков: 

    каждый узел списка описывает непрерывный свободный участок;
    узлы списка хранятся в начале каждого свободного блока.

\*) сначала у нас список из одного элемента (весь распределяемый участок лог.памяти), сам список хранится в начале участка памяти.

### <a id='toc6_2_1_'></a>[Аллокация свободного блока](#toc0_)

Пройдемся по списку и найдем блок достаточного размера:

    если блок слишком большой, то отрезаем от него часть; 
        в конце свободного участка выделяем size байт;
        перед ним в ячейку памяти заносим величину size;
        возвращаем указатель на начало участка размером size (после записи size).
    если походящего блока не нашлось, то возвращаем ошибку.

### <a id='toc6_2_2_'></a>[Освобождение занятого блока](#toc0_)

Чтобы освободить свободный блок, его нужно вернутьв список
    
    например, можно добавить в список новый элемент
        если так делать постоянно, то получится длинная структура из большого числа, без участков большого размера (фрагментация памяти)

### <a id='toc6_2_3_'></a>[Боремся с фрагментацией](#toc0_)
Варианты:

    тупо искать смежные блоки проходом по списку и объединять их (O(N));
    поддерживать список упорядоченным, объединять последовательные блоки (O(N));
    использовать упорядоченную структуру вместо списка, например сбалансированное дерево (AVL, RB...) (O(logN));
    есть решение лучше - использовать граничные маркеры (Border/Boundary Tags, O(1)).

### <a id='toc6_2_4_'></a>[Граничные маркеры](#toc0_)

Не только добавляется size перед блоком, но и перед и после блока добавляются метаданные (size + свободный/занятый блок). Тогда:

    при обсвобождении блока проверяется признак свободности блока перед освобождаемым,
        если свободен - объединить с текущим
        (рекурсивно повторять не нужно, т.к. остальные блоки уже будут максимально возможно объединены при собственном освобождении)
    аналогично проверить следующий за освобождаемым блок
        если свободен - объединить с текущим

# <a id='toc7_'></a>[Задача](#toc0_)

Попробуйте реализовать динамический аллокатор памяти (интерфейс в шаблоне кода). Введем несколько обозначений:

    BufSize - размер участка логической памяти, который ваш аллокатор будет распределять
    MaxSize - наибольший размер участка памяти, который можно аллоцировать вашим аллокатором для данного BufSize (проверяющая система найдет его бинарным поиском)
    EffectiveSize - максимальное количество памяти, которое может быть выделено пользователю для данного BufSize (например, если мы аллоцируем много небольших участков памяти)

Ваш аллокатор памяти должен удовлетворять следующим условиям:

    MaxSize должен быть не меньше 8/9 BufSize
    EffectiveSize должен быть не меньше 1/9 BufSize
    ваш аллокатор должен бороться с фрагментацией, т. е. если от начального состояния аллокатора мы смогли успешно аллоцировать какое-то количество памяти, то если мы 
    освободим всю эту память и заново попробуем повторить аллокацию, она должна быть успешной
    если аллокатор не смог аллоцировать участок памяти нужного размера, то он должен вернуть NULL
    использование динамической аллокации памяти запрещено (malloc/new/new[]/free/delete/delete[]).


Гарантируется

    что BufSize будет не меньше 100Kb и не больше 1Mb
    что минимальный аллоцируемый участок памяти будет не меньше 16 байт.



    $ time /home/user1/projects/stepik.org/OS/2.Memory/myallocator
    Allocable mem_size: 608148
    test  1: ( mem full )  15204 blocks of   16 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  2: ( mem full )  10860 blocks of   32 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  3: ( mem full )   6911 blocks of   64 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  4: ( mem full )   5068 blocks of   96 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  5: ( mem full )   4001 blocks of  128 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  6: ( mem full )   2172 blocks of  256 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  7: ( mem full )   1135 blocks of  512 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  8: ( mem full )    581 blocks of 1024 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test  9: ( mem full )    148 blocks of 4096 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100
    test 10: ( mem full )     75 blocks of 8192 bytes => ( mem free ) Fragments=    1, MaxSize= 608124, EffectiveSize= 608100

    real    0m0,011s
    user    0m0,007s
    sys     0m0,004s


простой и сравнительно универсальный подход к аллокации памяти

## <a id='toc7_1_'></a>[Buddy аллокатор](#toc0_)

Buddy аллокатор предназначен для аллокации больших участков памяти
    
    buddy аллокатор аллоцирует память блоками;
    блок - 2**i последовательных страниц;
    например, 1 страница, 2 страницы, 4 страницы и т. д.;
    но не 3 страницы или 5 страниц

*) 'страницы' бадди-аллокатора не обязаны совпадать с физическими/логическими. Это другие страницы. Но если совпадают, то кое какой выигрыш будет: у buddy аллокатора в его классическом виде аллокация идет блоками по степеням двойки. Если использовать buddy аллокатор для аллокации физических страниц, и использовать страницы совпадающими со страницами paging-а, то это ограничение можно легко приодолеть не изменяя собственно сам buddy аллокатор.  

### <a id='toc7_1_1_'></a>[Дескрипторы страниц](#toc0_)

Buddy аллокатор не будет работать с памятью напрямую

    вместо страниц памяти будем использовать в алгоритме дескрипторы;
    просто массив дескрипторов - по адресу страницы легко получить дескриптор и наоборот.

NB процессор при работе обращается к логическим адресам, которые преобразует в физические по установленному алгоритму (paging). Читать/писать в физ. память для которой нет какого-то отображения он не может. Тут на помощь приходит бадди аллокатор, он работает не с памятью, а с дескрипторами и писать в память, которую он аллоцирует ему не нужно. 

С помощью бадди аллокатора можно аллоцировать все что угодна (место на диске). Аллоцировать, значит зарезервировать и проверять валидность обращений. Дескрпторы это деталь реализации алгоритма, соответсвенно реализация и должна позаботиться о том, что они существует и соответсвенно реализация должна предусмотреть способ доступа к ним.

Что будет храниться в дескрипторах?

    указатели, чтобы связать дескрипторы в двусвязный список;
    признак свободности/занятости;
    уровень (указание на размер блока), т.е. показатель степени 2.

### <a id='toc7_1_2_'></a>[Списки свободных блоков](#toc0_)

Пусть у нас изначально есть 2\*\*N последовательных свободных страниц:

    заведем N+1 изначально пустой двусвязный список;
    i-ый список будет хранить свободные блоки размером 2**i страниц.

Начальное состояние

    Возьмем дескриптор 0-ой страницы
    отметим дескриптор как свободный;
    зададим в дескрипторе уровень N−1;
    добавим дескриптор в список N−1.
    
### <a id='toc7_1_3_'></a>[Инициализация](#toc0_)

    Пусть нам нужен блок памяти размером 2\*\*3 = 8 страниц.
    0й десткриптор обозначаем как свободный, устанавливаем уровень 3 (т.е. блок этого дескриптора представляет весь блок из 8 страниц)

### <a id='toc7_1_4_'></a>[Аллокация](#toc0_)

Мы хотим аллоцировать 2\*\*i страниц

    найдем непустой список с блоками ≥ 2**i;
    берем один из блоков и делим его пополам, пока неостанется 2**i (допустим i=0 т.е. нужен минимальный блок размером в 1 страницу):
        уровень 0 дескриптора в списке понижается на 1 (теперь он представляет 4 страницы), 
            это значит что он удалается из списка дескрипторов уровня 3 и заносится в список дескрипторов уровня 2
        создается (временный) дескриптор уровня 2 (тоже на 4 страницы), он то и делится дальше:
            первый дескриптор с уровнем 1 заносится в список дескрипторов уровня 1
            второй с уровнем 1 делится дальше:
                первый дескриптор идет в список уровня 0
                второй подходит по размеру
                    метим занятым 
                    заносим в список уровня 0
                    возвращаем дескриптор
                
                
### <a id='toc7_1_5_'></a>[Освобождение](#toc0_)

Теперь мы хотим освободить блок размера 2\*\*i

    мы могли бы просто добавить дескриптор в список i, но это приведет к фрагментации;
    мы должны попытаться объединить смежные блоки:
        проверить свободен ли парный (buddy) блок того же уровня    
            если да, то объеденить и поднять на уровнь выше

Buddies. Как найти парный блок?

    если мы осовобождаем блок размера 2**i с номером j;
    парный блок имеет номер j⊕2**i;
    ⊕- исключющее побитовое ИЛИ ^.

*) суть формулы заключается в том, чтобы изменить i-ый бит в j на противоположный, для этого нужно xor-ить не с i, а с 2^i

Освобождение. Когда можно объединять парные блоки?

    если оба блока свободны;
    если оба блока имеют один размер (уровень в дескрипторе).
        нужно проверять размер, иначе мы попытаемя объединить с бадди блоком у которого есть свой занятый бадди его же размера






## <a id='toc7_2_'></a>[Кеширующий аллокатор.](#toc0_)

Создадим кеш блоков фиксированного размера

    кеш будет аллоцировать/освобождать большие регионы памяти, используя другой аллокатор (например бадди);
    кеш "нарезает" большие регионы на блоки фиксированного размера (внутри блоки могут быть фрагментированы).

Кеширующий аллокатор имеет ряд достоинств:

    фиксированный размер блоков позволяет бороться с фрагментацией;
    аллокация/освобождение могут работать за O(1);
    можно с комбинировать кеши разных размеров и построить универсальный аллокатор.

# <a id='toc8_'></a>[ SLAB аллокатор](#toc0_)

Кеширующий (аллоцирует куски одного размера) аллокатор, предложенный Джеффом Бонвиком и использованный в SunOS (Solaris). slab - плитка, сляб:

    slab - описывает большой регион памяти, который разбивается на маленькие блоки фиксированного размера;
    все свободные блоки связываются в список;
    количество элементов списка и указатель на первый элемент сохраняются в заголовке:
        в конце сляба (эмпирически установлено, что так удобнее) размещается заголовок, который содержит 
            количество свободных элементов и указатель на первый свободный элемент.
    
    *) под объекты разного размера, системе системе нужно создавать SLAB аллокатор под каждый размер
    *) тонкий момент в том, что размер сляба в слаб-аллокаторе подбирается под задачу. Можно пихнуть все в один сляб, увеличив 
        время доступа за счет длинного списка, или распихать по большому количеству маленьких слябов, увеличив количество 
        памяти на метаданные заголовков
    
SLAB аллокатор управляет slab-ами:

    если нет slab-а со свободными объектами - аллоцируем новый;
    если все объекты в slab-е свободны - можно освободить slab.

SLAB аллокатор поддерживает три списка slab-ов:
    
    полностью свободные slab-ы (как можно быстрее возвращается системе);
    частично занятые slab-ы (которые можно использовать);
    полностью занятые slab-ы (коротые не нужно просматривать для аллокации).
        
        если список частично занятых слябов пустой, смотрим
            если список полностью свободных слябок пустой,
                аллоцируем новый сляб, выдаем из него
            иначе
                выдаем из него
        иначе
            выдаем из него

Чтобы освободить элемент, нужно найти slab, которому он принадлежит

    мы можем сохранить указатель на slab рядом с аллоцированной памятью (как в простом подходе к аллокации);
    мы можем потребовать, чтобы размер и выравнивание slab-а были равны 2**i:
        т.е. если размер сляба k*2**i, то тогда битовая арифметика с указателем на аллоцированный кусок памяти:
            PTR & ~(2**i - 1) // у маски будет i последних бит равны 1, их инвертируем ~, 
            побитовое И такой маски c указателем занулит последние i бит в адресе, т.е. оставшиеся старшие биты указателя это 
            адрес начала сляба

            в нем при обработке free(PTR) уменьшаем счетчик занятых блоков
            


## <a id='toc8_1_'></a>[Заметки по SLAB-аллокатору](#toc0_)

SLOB allocator (1991-1999) K&R allocator (чето типа нашего предыдущего примера простого подхода к аллокации)  
SLAB allocator (1999-2008) Solaris type allocator  
SLUB (unqueued) allocator (2008 - today)  
